In [4]:
import pandas as pd
import numpy as np
from scipy.special import gammaincc  # igamc

def approximate_entropy_test(sequence: str, m: int):
    n = len(sequence)

    def get_block_frequencies(seq, block_size):
        padded_seq = seq + seq[:block_size - 1]  # extensão cíclica
        counts = {}
        for i in range(n):
            block = padded_seq[i:i + block_size]
            counts[block] = counts.get(block, 0) + 1
        return counts

    def calculate_phi(freq_counts):
        total = sum(freq_counts.values())
        phi = 0.0
        for count in freq_counts.values():
            p = count / total
            phi += p * np.log(p)
        return phi

    freq_m = get_block_frequencies(sequence, m)
    freq_m1 = get_block_frequencies(sequence, m + 1)

    phi_m = calculate_phi(freq_m)
    phi_m1 = calculate_phi(freq_m1)

    ApEn = phi_m - phi_m1
    chi_squared = 2.0 * n * (np.log(2) - ApEn)
    p_value = gammaincc(2 ** (m - 1), chi_squared / 2.0)

    return ApEn, chi_squared, p_value

# Caminho do arquivo
file_path = "seqbinaria_participantes.csv"

try:
    df = pd.read_csv(file_path, sep="\t", header=None, names=['Candidato', 'Mediana', 'Sequencia'])

    results = []

    for _, row in df.iterrows():
        seq_str = row['Sequencia']
        seq_len = len(seq_str)
        seq_binaria = np.array(list(seq_str), dtype=int)
        n_1s = np.sum(seq_binaria)
        n_0s = seq_len - n_1s

        ApEn, chi2, p_val = approximate_entropy_test(seq_str, m=2)

        if p_val >= 0.01:
            interpretacao = "Alta aleatoriedade"
        elif 0.001 <= p_val < 0.01:
            interpretacao = "Aleatoriedade moderada"
        else:
            interpretacao = "Baixa aleatoriedade"

        results.append({
            'candidato': row['Candidato'],
            'mediana': row['Mediana'],
            'tamanho_sequencia': seq_len,
            'n_1s': n_1s,
            'n_0s': n_0s,
            'ApEn': round(ApEn, 6),
            'chi_squared': round(chi2, 6),
            'p_valor': round(p_val, 6),
            'sequencia': seq_str,
            'interpretacao': interpretacao
        })

    df_results = pd.DataFrame(results)

    # Geração do resumo estatístico
    resumo = df_results['interpretacao'].value_counts().reset_index()
    resumo.columns = ['Interpretação', 'Quantidade']
    resumo['Faixa de p-valor'] = resumo['Interpretação'].map({
        'Alta aleatoriedade': 'p ≥ 0,01',
        'Aleatoriedade moderada': '0,001 ≤ p < 0,01',
        'Baixa aleatoriedade': 'p < 0,001'
    })
    resumo['Percentual (%)'] = (resumo['Quantidade'] / len(df_results) * 100).round(2)

    # Ordena conforme prioridade lógica
    ordem = ['Alta aleatoriedade', 'Aleatoriedade moderada', 'Baixa aleatoriedade']
    resumo['ordem'] = resumo['Interpretação'].apply(lambda x: ordem.index(x))
    resumo = resumo.sort_values('ordem').drop(columns='ordem')
    resumo = resumo[['Interpretação', 'Faixa de p-valor', 'Quantidade', 'Percentual (%)']]

    # Exibe no console
    print(resumo.to_string(index=False))

    # Exporta os resultados
    df_results.to_csv("resultado_entropia_aproximada.csv", index=False)

except FileNotFoundError:
    print("Arquivo não encontrado.")


     Interpretação Faixa de p-valor  Quantidade  Percentual (%)
Alta aleatoriedade         p ≥ 0,01          55           100.0
